In [12]:
## 1. DataSet import
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from prepare_datasets import *
from Helper_functions import *

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

X, y, feature_names, categorical_features, continuous_features, actionable_features = get_and_prepare_german_dataset()

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X_pos = X[y == 1]
X_neg = X[y == 0]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [13]:
from Model import NeuralNetwork

model = NeuralNetwork(X.shape[1], 200, 2)
model1= NeuralNetwork(X.shape[1], 200, 2)
model2= NeuralNetwork(X.shape[1], 200, 2)
model3= NeuralNetwork(X.shape[1], 200, 2)
model4= NeuralNetwork(X.shape[1], 200, 2)
model5= NeuralNetwork(X.shape[1], 200, 2)
model6= NeuralNetwork(X.shape[1], 200, 2)

In [14]:
models = [model,model1, model2, model3, model4, model5, model6]
lambdas = [0,0.05,0.1,0.15,0.2,0.25,0.3]

model_path = f"models/Model_0.pth"
model.load_state_dict(torch.load(model_path))
model.eval()  # Set to evaluation mode

# Load saved weights
for lambda_model, lamda in zip(models[1:], lambdas[1:]):
    model_path = f"models/model_lambda_{lamda:.2f}.pth"
    lambda_model.load_state_dict(torch.load(model_path))
    lambda_model.eval()  # Set to evaluation mode


In [15]:
import torch.nn.functional as F
class WrappedModelForAlibi:
    def __init__(self, model):
        self.model = model
        self.model.eval()  # Important for consistent behavior

    def predict(self, x):
        with torch.no_grad():
            x_tensor = torch.tensor(x, dtype=torch.float32)
            logits = self.model(x_tensor)

            if logits.ndim == 1:
                logits = logits.unsqueeze(0)  # Ensure shape is (1, num_classes)

            probs = F.softmax(logits, dim=1)
            return probs.numpy()


In [16]:
from alibi.explainers import CEM
X_np = X.numpy()

# Wrap your model
predict_fn = WrappedModelForAlibi(model).predict

# Feature ranges from training data
feature_min = X_np.min(axis=0)
feature_max = X_np.max(axis=0)
feature_range = (feature_min, feature_max)

cem = CEM(
    predict_fn,
    mode='PN',
    shape=(1, X_np.shape[1]),
    max_iterations=100,
    feature_range = feature_range
)
# Fit on a sample of training data (preferably more than 1)
cem.fit(X_np)

# Explain a sample (batch size 1)
explanation = cem.explain(X_np[1:2])

print(explanation)


Explanation(meta={
  'name': 'CEM',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'mode': 'PN',
              'shape': (1, 27),
              'kappa': 0.0,
              'beta': 0.1,
              'feature_range': (array([  0.,   0.,   0.,  19.,   4., 250.,   1.,   1.,   1.,   1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.], dtype=float32), array([1.0000e+00, 1.0000e+00, 1.0000e+00, 7.5000e+01, 7.2000e+01,
       1.8424e+04, 4.0000e+00, 4.0000e+00, 4.0000e+00, 2.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
       1.0000e+00, 1.0000e+00], dtype=float32)),
              'gamma': 0.0,
              'learning_rate_init': 0.01,
              'max_iterations': 100,
              'c_init': 10.0,
      

In [17]:
import torch

# Original instance and counterfactual from Alibi
original = torch.tensor(explanation.data['X'][0], dtype=torch.float32)
cf = torch.tensor(explanation.data['PN'][0], dtype=torch.float32)

# L1 distance
distance = torch.norm(original - cf, p=1)

print("L1 Distance between original and PN:", distance.item())

L1 Distance between original and PN: 3.26816725730896


In [ ]:
from tqdm import tqdm

X_false_negatives, X_true_negatives, _, _  = split_by_classification(model,X_neg)

l1_distances = []
l2_distances = []

# Loop over each instance
for i in tqdm(range(len(X_true_negatives)), desc="Generating CEM counterfactuals"):
    x = X_true_negatives[i:i+1]  # keep shape (1, n_features)
    explanation = cem.explain(x)

    # Extract original and PN (pertinent negative) instance
    original = explanation.data['X'][0]
    pn = explanation.data['PN']

    if pn is not None:
        cf = pn[0]
        l1_distance = torch.norm(original - cf, p=1).item()
        l2_distance = torch.norm(original - cf, p=2).item()
        l1_distances.append(l1_distance)
        l2_distances.append(l2_distance)

l1_distances = np.array(l1_distances)
l2_distances = np.array(l2_distances)
# Print stats
print("Mean L1 Distance:", np.mean(l1_distances))
print("Mean L2 Distance:", np.mean(l2_distances))

Generating CEM counterfactuals:   0%|          | 0/122 [00:00<?, ?it/s]C:\Users\hamma\AppData\Local\Temp\ipykernel_2532\218918381.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tensor = torch.tensor(x, dtype=torch.float32)
Generating CEM counterfactuals:   2%|▏         | 3/122 [00:13<09:11,  4.63s/it]

In [55]:
# fist we calculate this subset
_, commun_negatives, _, _ = split_by_classification(model, X_neg)

for current_model in models:
    _, commun_negatives, _, _ = split_by_classification(current_model, commun_negatives)

print(len(commun_negatives))

122


In [57]:
cost_of_recourses_l1 = []
cost_of_recourses_l2 = []

for (best_model, lamda ) in zip(models,lambdas):
    print(f"Evaluating using CEM with lambda = {lamda}")
    X_np = X.numpy()

    predict_fn = WrappedModelForAlibi(best_model).predict
    # Feature ranges from training data
    feature_min = X_np.min(axis=0)
    feature_max = X_np.max(axis=0)
    feature_range = (feature_min, feature_max)

    cem = CEM(
        predict_fn,
        mode='PN',
        shape=(1, X_np.shape[1]),
        max_iterations=100,
        feature_range = feature_range
    )
    # Fit on a sample of training data (preferably more than 1)
    cem.fit(X_np)


    l1_distances = []
    l2_distances = []

    # Loop over each instance
    for i in tqdm(range(len(commun_negatives)), desc="Generating CEM counterfactuals"):
        x = commun_negatives[i:i+1]  # keep shape (1, n_features)
        explanation = cem.explain(x)

        # Extract original and PN (pertinent negative) instance
        original = explanation.data['X'][0]
        pn = explanation.data['PN']

        if pn is not None:
            cf = pn[0]
            l1_distance = torch.norm(original - cf, p=1).item()
            l2_distance = torch.norm(original - cf, p=2).item()
            l1_distances.append(l1_distance)
            l2_distances.append(l2_distance)

    l1_distances = np.array(l1_distances)
    l2_distances = np.array(l2_distances)

    # Compute mean L1 distance
    cost_of_recourses_l1.append(l1_distances)
    cost_of_recourses_l2.append(l1_distances)

    print(f"Mean L1 distance for negatively classified data using DiCE: {np.mean(l1_distances):.2f}")
    print(f"Mean L2 distance for negatively classified data using DiCE: {np.mean(l2_distances):.2f}")



Evaluating using CEM with lambda = 0.05


Generating CEM counterfactuals:   0%|          | 0/122 [00:00<?, ?it/s]C:\Users\hamma\AppData\Local\Temp\ipykernel_14952\187860461.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tensor = torch.tensor(x, dtype=torch.float32)
Generating CEM counterfactuals: 100%|██████████| 122/122 [10:26<00:00,  5.14s/it]


Mean L1 distance for negatively classified data using DiCE: 2.83
Mean L2 distance for negatively classified data using DiCE: 1.34
Evaluating using CEM with lambda = 0.1


Generating CEM counterfactuals:  36%|███▌      | 44/122 [03:24<06:03,  4.66s/it]


KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 6))
plt.boxplot(cost_of_recourses_l1,patch_artist=True, tick_labels=lambdas)
plt.title("Cost of Recourse Across Models (L1 Distance)")
plt.xlabel("Lambda / Model Index")
plt.ylabel("L1 Distance")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.boxplot(cost_of_recourses_l2,patch_artist=True, tick_labels=lambdas)
plt.title("Cost of Recourse Across Models (L2 Distance)")
plt.xlabel("Lambda / Model Index")
plt.ylabel("L2 Distance")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
l1_means = [np.mean(sublist) for sublist in cost_of_recourses_l1]
## Plotting of the mean of Recourse genereated by dice After the Training with different Lambdas
plt.figure(figsize=(5, 5))
plt.plot(lambdas, l1_means, marker='o', color='red')
plt.title("Mean Recourse calculated by DiCE vs Lambda")
plt.xlabel("Lambda")
plt.ylabel("Mean Recourse ")
plt.grid(True)


l2_means = [np.mean(sublist) for sublist in cost_of_recourses_l2]
## Plotting of the mean of Recourse genereated by dice After the Training with different Lambdas
plt.figure(figsize=(5, 5))
plt.plot(lambdas, l2_means, marker='o', color='red')
plt.title("Mean Recourse calculated by DiCE vs Lambda")
plt.xlabel("Lambda")
plt.ylabel("Mean Recourse ")
plt.grid(True)
